In [ ]:
import requests
from datetime import datetime
import json
import time
import pandas as pd

def fetch_cme_data(start_year, end_year, api_key):
    all_data = []

    for year in range(start_year, end_year + 1):
        start = f"{year}-01-01"
        end = f"{year}-12-31"

        print(f"Fetching data for: {start} to {end}")
        params = {
            'startDate': start,
            'endDate': end,
            'api_key': api_key
        }

        try:
            response = requests.get('https://api.nasa.gov/DONKI/CME', params=params)

            if response.status_code == 200:
                data = response.json()
                all_data.extend(data)
                print(f"Retrieved {len(data)} entries for {year}")
            else:
                print(f"Error {response.status_code} for year {year}")
        except requests.exceptions.RequestException as e:
            print(f"Request failed for year {year}: {e}")

        # Optional: Pause to avoid rate-limiting
        time.sleep(1)

    return all_data

# Call the function
api_key = 'aeRc3h5tR5wweegUs1yyQ98QGs1H4ti2jpGW5hGn'
cme_data = fetch_cme_data(2009, 2024, api_key)



Fetching data for: 2009-01-01 to 2009-12-31
Retrieved 0 entries for 2009
Fetching data for: 2010-01-01 to 2010-12-31
Retrieved 44 entries for 2010
Fetching data for: 2011-01-01 to 2011-12-31
Retrieved 231 entries for 2011
Fetching data for: 2012-01-01 to 2012-12-31
Retrieved 354 entries for 2012
Fetching data for: 2013-01-01 to 2013-12-31
Retrieved 357 entries for 2013
Fetching data for: 2014-01-01 to 2014-12-31
Retrieved 691 entries for 2014
Fetching data for: 2015-01-01 to 2015-12-31
Retrieved 523 entries for 2015
Fetching data for: 2016-01-01 to 2016-12-31
Retrieved 333 entries for 2016
Fetching data for: 2017-01-01 to 2017-12-31
Retrieved 222 entries for 2017
Fetching data for: 2018-01-01 to 2018-12-31
Retrieved 51 entries for 2018
Fetching data for: 2019-01-01 to 2019-12-31
Retrieved 66 entries for 2019
Fetching data for: 2020-01-01 to 2020-12-31
Retrieved 143 entries for 2020
Fetching data for: 2021-01-01 to 2021-12-31
Retrieved 510 entries for 2021
Fetching data for: 2022-01-01 

In [34]:
from pprint import pprint
print(len(cme_data))
pprint(cme_data[0])

7296
{'activeRegionNum': None,
 'activityID': '2010-04-03T09:54:00-CME-001',
 'catalog': 'M2M_CATALOG',
 'cmeAnalyses': [{'enlilList': [{'au': 2.0,
                                 'cmeIDs': ['2010-04-03T09:54:00-CME-001'],
                                 'estimatedDuration': 4.2,
                                 'estimatedShockArrivalTime': '2010-04-05T10:37Z',
                                 'impactList': [{'arrivalTime': '2010-04-05T19:00Z',
                                                 'isGlancingBlow': False,
                                                 'location': 'STEREO A'},
                                                {'arrivalTime': '2010-04-07T11:00Z',
                                                 'isGlancingBlow': True,
                                                 'location': 'Mars'}],
                                 'isEarthGB': False,
                                 'kp_135': 5,
                                 'kp_18': None,
                         

In [23]:
df = pd.DataFrame(cme_data)

# Replace NaNs with None for valid JSON
df = df.where(pd.notnull(df), None)
df = df.fillna('Unknown')
df['Date'] = pd.to_datetime(df['startTime'], errors='coerce', format='ISO8601').dt.strftime('%Y-%m-%d')
# Convert to a list of dictionaries
data_to_save = df.to_dict(orient='records')

# Save to JSON
local_file_path = "../Resources/CME_Data.json"
try:
    with open(local_file_path, 'w') as f:
        json.dump(data_to_save, f, indent=4)
    print(f"Cleaned data successfully saved to {local_file_path}")
except Exception as e:
    print(f"Error saving to local file: {e}")

Cleaned data successfully saved to ../Resources/CME_Data.json


In [8]:
df.head(1)

,activityID,catalog,startTime,instruments,sourceLocation,activeRegionNum,note,submissionTime,versionId,link,cmeAnalyses,linkedEvents,Date
0,2010-04-03T09:54:00-CME-001,M2M_CATALOG,2010-04-03T09:54Z,"[{'displayName': 'STEREO A: SECCHI/COR2'}, {'d...",S20E05,Unknown,SDO images were unavailable. CME source locati...,2015-06-17T19:47Z,1,https://webtools.ccmc.gsfc.nasa.gov/DONKI/view...,"[{'isMostAccurate': True, 'time21_5': '2010-04...",[{'activityID': '2010-04-03T09:04:00-FLR-001'}...,2010-04-03


In [14]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

df_model = pd.DataFrame(cme_data)

# Define the spherical to cartesian conversion function
def spherical_to_cartesian(lat, lon, radius=1):
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    x = radius * np.cos(lat_rad) * np.cos(lon_rad)
    y = radius * np.cos(lat_rad) * np.sin(lon_rad)
    z = radius * np.sin(lat_rad)
    return x, y, z

# Example: extract lat/lon from nested 'cmeAnalyses' list of dicts
df['latitude'] = df['cmeAnalyses'].apply(lambda x: x[0]['latitude'] if x and isinstance(x, list) and 'latitude' in x[0] else None)
df['longitude'] = df['cmeAnalyses'].apply(lambda x: x[0]['longitude'] if x and isinstance(x, list) and 'longitude' in x[0] else None)
df['speed'] = df['cmeAnalyses'].apply(lambda x: x[0]['speed'] if x and isinstance(x, list) and 'speed' in x[0] else None)
df['year'] = pd.to_datetime(df['startTime']).dt.year
# Drop rows where lat or lon is missing
df_clean = df.dropna(subset=['latitude', 'longitude'])

# Compute X, Y, Z coordinates
df_clean['x'], df_clean['y'], df_clean['z'] = zip(*df_clean.apply(
    lambda row: spherical_to_cartesian(row['latitude'], row['longitude']), axis=1
))

df_clean.head(1)

# Unique years
years = sorted(df_clean['year'].dropna().unique())

# Create a trace for each year
data_traces = []
buttons = []

for idx, year in enumerate(years):
    df_year = df_clean[df_clean['year'] == year]
    trace = go.Scatter3d(
        x=df_year['x'],
        y=df_year['y'],
        z=df_year['z'],
        mode='markers',
        marker=dict(
            size=6,
            color=df_year['speed'],
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title='Speed (km/s)') if idx == 0 else None
        ),
        name=str(year),
        text=df_year['activityID'],
        hoverinfo='text',
        visible=(idx == 0)  # Show only the first year initially
    )
    data_traces.append(trace)

    # Create button for this year
    buttons.append(dict(
        label=str(year),
        method='update',
        args=[{'visible': [i == idx for i in range(len(years))]},
              {'title': f'CME Events for {year}'}]
    ))


# Create the figure
fig = go.Figure(data=data_traces)

# Add dropdown menu
fig.update_layout(
    updatemenus=[dict(
        active=0,
        buttons=buttons,
        direction='down',
        showactive=True,
        x=0.1,
        xanchor='left',
        y=1.15,
        yanchor='top'
    )],
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='data'
    ),
    title=f'CME Events for {years[0]}',
    margin=dict(l=0, r=0, b=0, t=30)
)

fig.show()



C:\Users\12087\AppData\Local\Temp\ipykernel_38592\1770633696.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\12087\AppData\Local\Temp\ipykernel_38592\1770633696.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\12087\AppData\Local\Temp\ipykernel_38592\1770633696.py:25: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [ ]:
list_of_impact = []

for cme_event in cme_data:
    for analysis in cme_event.get('cmeAnalyses', []):
        for enlil in analysis.get('enlilList', []):
            impact_list = enlil.get('impactList')
            if impact_list is not None:
                for impact in impact_list:
                    if impact.get('location') is not None:
                        list_of_impact.append({
                            'activityID': cme_event.get('activityID'),
                            'arrivalTime': impact.get('arrivalTime'),
                            'isGlancingBlow': impact.get('isGlancingBlow')
                        })

# Optional: print the results
if list_of_impact:
    print("Earth-impacting CMEs:")
    for i in list_of_impact:
        print(i)
else:
    print("No Earth impacts found.")


No Earth impacts found.
